<a href="https://colab.research.google.com/github/VitikaJain25/NLP/blob/master/NLP_Project_Review_Classification_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer

import pickle

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Data Exploration

In [3]:
data = pd.read_excel("/content/Canva_reviews.xlsx")

In [4]:
data.shape

(1500, 13)

In [5]:
data.head()

,reviewId,userName,userImage,review,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,Sentiment,Sub Category,Sub Category_test
0,gp:AOqpTOFxf3fttcT5DSvFIn9KPp5FErgH9yC533Fmoxv...,Donna Caritero,https://play-lh.googleusercontent.com/a-/AOh14...,Overall it's really an amazing app. I've been ...,4,528,2.116.0,2021-06-17 07:18:54,Hi Donna. We are sorry that your text or desig...,2021-06-19 21:24:32,Negative,NaN,bug_picture_quality
1,gp:AOqpTOEq6rNIWLnPV4KFTctWvm0mpGEQljtD6mvy1H-...,Soumi Mukhopadhyay,https://play-lh.googleusercontent.com/a-/AOh14...,Hey! Yes I gave a 5 star rating... coz I belie...,5,351,2.116.0,2021-06-17 19:18:28,NaN,NaT,Positive,extremely_happy,NaN
2,gp:AOqpTOE86hSyPRHZgYt28Uk5zGe4FZGb1hkmtFDiYJ2...,Theknown _unknown,https://play-lh.googleusercontent.com/a-/AOh14...,Canva used to be a good app! But recently I've...,1,160,2.116.0,2021-06-23 19:13:28,Hi there. We're sorry to hear that you are hav...,2021-06-26 20:20:56,Negative,saving_downloading_issue,bug_saving_files
3,gp:AOqpTOHSuKkVTcM3QgCCKysHQlxEnk2ocOKsUMiMIJy...,Anthony Dean,https://play-lh.googleusercontent.com/a/AATXAJ...,"It's a brilliant app, but I have just one prob...",5,145,2.116.0,2021-06-19 23:36:07,NaN,NaT,Negative,others,bug_app_crash
4,gp:AOqpTOEOrZt5H6jXPiplJyffCd5ZBnVXACTWgwNsF1R...,Neha Diana Wesley,https://play-lh.googleusercontent.com/a-/AOh14...,This was such a great app. I used to make BTS ...,4,142,2.116.0,2021-06-21 00:29:27,Hi Neha. Sorry to hear that you are unable to ...,2021-06-24 20:46:28,Negative,others,bug_other


In [6]:
data.columns

Index(['reviewId', 'userName', 'userImage', 'review', 'score', 'thumbsUpCount',
       'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'Sentiment',
       'Sub Category', 'Sub Category_test'],
      dtype='object')

In [7]:
data["Sentiment"].value_counts()

Positive    1032
Negative     468
Name: Sentiment, dtype: int64

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              1500 non-null   object        
 1   userName              1500 non-null   object        
 2   userImage             1500 non-null   object        
 3   review                1500 non-null   object        
 4   score                 1500 non-null   int64         
 5   thumbsUpCount         1500 non-null   int64         
 6   reviewCreatedVersion  1500 non-null   object        
 7   at                    1500 non-null   datetime64[ns]
 8   replyContent          523 non-null    object        
 9   repliedAt             523 non-null    datetime64[ns]
 10  Sentiment             1500 non-null   object        
 11  Sub Category          1499 non-null   object        
 12  Sub Category_test     23 non-null     object        
dtypes: datetime64[ns](

In [9]:
data.describe()

,score,thumbsUpCount
count,1500.000000,1500.000000
mean,4.154000,1.591333
std,1.361195,18.936791
min,1.000000,0.000000
25%,4.000000,0.000000
50%,5.000000,0.000000
75%,5.000000,0.000000
max,5.000000,528.000000


In [10]:
data = data[["review", "Sentiment"]]

## Convert text to lower case

In [11]:
reviews = list(data["review"])

In [12]:
reviews_lower = [rev.lower() for rev in reviews]

# Tokenization

In [13]:
# Applying word tokenizer to every review in out dataset.
tokens = [word_tokenize(rev) for rev in reviews_lower]

In [ ]:
# Option 2 - usinf apply method.
# def lower_sent(text):
#   return text.lower()

# data["lower_sent"] = data["review"].apply(lower_sent)

## Remove Stopwords

In [15]:
stop_words = stopwords.words("english")

In [ ]:
# Method 1

# for i in range(len(tokens)):
#   tokens_no_stop_word = [word for word in tokens[i] if word not in stop_words]
#   # print(tokens_no_stop_word)
#   tokens[i] = " ".join(tokens_no_stop_word)

In [16]:
# Method 2
# Loop over sub list and main list aswell.
tokens = [[word for word in t if word not in stop_words] for t in tokens]

## Remove Punctuations

In [17]:
# usinf Regex Tokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [18]:
tokens = [["".join(tokenizer.tokenize(word)) for word in t
           if len(tokenizer.tokenize(word)) > 0] for t in tokens]

## Stemming

In [19]:
porter = PorterStemmer()

In [20]:
# Applying Stemming to all tokens. To reduce number of words.
tokens = [[porter.stem(word) for word in t] for t in tokens]

# Word Count Distribution

In [21]:
flat_tokens = [word for t in tokens for word in t]

In [22]:
len(flat_tokens)

13782

In [23]:
flat_tokens[:10]

['overal', 's', 'realli', 'amaz', 'app', 've', 'use', 'past', '5', 'year']

In [24]:
counts = Counter(flat_tokens)

In [25]:
len(counts)

1720

In [26]:
counts.most_common(10)

[('app', 932),
 ('use', 474),
 ('design', 349),
 ('easi', 293),
 ('s', 240),
 ('love', 220),
 ('good', 219),
 ('edit', 191),
 ('make', 190),
 ('great', 184)]

# Combining tokens to form sentence.

In [27]:
# combining tokens to form sentence.
clean_reviews = [" ".join(t) for t in tokens]

# BOW (Bag of Words)

In [28]:
# For any word to be included in vocabulary, say it has to be present in 5 documents.
# We can have higher threshold if we have a larger dataset.
vect = CountVectorizer(binary = True, min_df = 5)

In [29]:
X = vect.fit_transform(clean_reviews)

In [30]:
# This means there are 413 tokens which satisfy min_df = 5.
X.shape

(1500, 413)

## Unique Words in Vocabulary

In [31]:
len(vect.vocabulary_)

413

In [33]:
data["Sentiment"] = data["Sentiment"].apply(lambda x: 1 if x == "Positive" else 0)

In [34]:
data.head()

,review,Sentiment
0,Overall it's really an amazing app. I've been ...,0
1,Hey! Yes I gave a 5 star rating... coz I belie...,1
2,Canva used to be a good app! But recently I've...,0
3,"It's a brilliant app, but I have just one prob...",0
4,This was such a great app. I used to make BTS ...,0


In [35]:
y = data["Sentiment"]

# Split Dataset into train and test

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 42)

# Build Logistic Regression Model

In [37]:
model = LogisticRegression()

In [38]:
model.fit(X_train, y_train)

LogisticRegression()

In [39]:
train_pred = model.predict(X_train)

In [40]:
accuracy_score(y_train, train_pred)

0.9608333333333333

In [41]:
test_pred = model.predict(X_test)

In [42]:
accuracy_score(y_test, test_pred)

0.89

# Save both the model and Vectorizer

In [43]:
with open("/content/binary_count_vect.pkl", "wb") as f:
  pickle.dump(vect, f)
with open("/content/binary_count_vect_lr.pkl", "wb") as f:
  pickle.dump(model, f)

# Bag Of Words without Binary

In [44]:
vect = CountVectorizer(min_df = 5)

In [45]:
X = vect.fit_transform(clean_reviews)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 42)

In [47]:
model = LogisticRegression()

In [48]:
model.fit(X_train, y_train)

LogisticRegression()

In [49]:
train_pred = model.predict(X_train)

In [50]:
test_pred = model.predict(X_test)

In [51]:
print(f"Train Accuracy:{accuracy_score(y_train, train_pred)}")
print(f"Test Accuracy:{accuracy_score(y_test, test_pred)}")

Train Accuracy:0.9566666666666667
Test Accuracy:0.8833333333333333


# N-grams

In [52]:
vect = CountVectorizer(min_df = 5, ngram_range = (1,3))

In [53]:
X = vect.fit_transform(clean_reviews)

In [54]:
X.shape

(1500, 666)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 42)

In [59]:
model = LogisticRegression()

In [60]:
model.fit(X_train, y_train)

LogisticRegression()

In [61]:
train_pred = model.predict(X_train)

In [62]:
test_pred = model.predict(X_test)

In [63]:
print(f"Train Accuracy:{accuracy_score(y_train, train_pred)}")
print(f"Test Accuracy:{accuracy_score(y_test, test_pred)}")

Train Accuracy:0.9716666666666667
Test Accuracy:0.8933333333333333


In [64]:
with open("/content/n_gram.pkl", "wb") as f:
  pickle.dump(vect, f)
with open("/content/n_gram_lr.pkl", "wb") as f:
  pickle.dump(model, f)

# TF-IDF

In [65]:
vect = TfidfVectorizer(min_df = 5)

In [66]:
X = vect.fit_transform(clean_reviews)

In [67]:
X.shape

(1500, 413)

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 42)

In [69]:
model = LogisticRegression()

In [70]:
model.fit(X_train, y_train)

LogisticRegression()

In [71]:
train_pred = model.predict(X_train)

In [72]:
test_pred = model.predict(X_test)

In [73]:
print(f"Train Accuracy:{accuracy_score(y_train, train_pred)}")
print(f"Test Accuracy:{accuracy_score(y_test, test_pred)}")

Train Accuracy:0.935
Test Accuracy:0.8766666666666667


In [74]:
with open("/content/tf-idf.pkl", "wb") as f:
  pickle.dump(vect, f)
with open("/content/tf-idf_lr.pkl", "wb") as f:
  pickle.dump(model, f)

# Prediction for new reviews

We have see that till now, the best model is n-gram.

So we will load the model.

In [75]:
# Sample test reviews
test_review_1 = """this is a truly amazing app , best for those who havw content but don't know how to express it
in a good and sharable manner. Thanks Team Canva for such a great app.
"""
test_review_2 = """Its the worst app ever I save my design lts not save"""

## Load the best model (n-gram) and its vectorizer

In [76]:
vect = pickle.load(open("/content/n_gram.pkl", "rb"))
model = pickle.load(open("/content/n_gram_lr.pkl", "rb"))

## Convert the reviews to a batch of one record

In [77]:
test_review_1 = [test_review_1]
test_review_2 = [test_review_2]

## Convert the text to lower case

In [78]:
test_review_1 = [r.lower() for r in test_review_1]
test_review_2 = [r.lower() for r in test_review_2]

## Tokenize the text

In [79]:
tokens_1 = [word_tokenize(r) for r in test_review_1]
tokens_2 = [word_tokenize(r) for r in test_review_2]

## Remove Stopwords

In [80]:
tokens_1 = [[word for word in t if word not in stop_words] for t in tokens_1]
tokens_2 = [[word for word in t if word not in stop_words] for t in tokens_2]

## Remove Punctuations

In [81]:
tokens_1 = [["".join(tokenizer.tokenize(word)) for word in t
             if len(tokenizer.tokenize(word))>0] for t in tokens_1]
tokens_2 = [["".join(tokenizer.tokenize(word)) for word in t
             if len(tokenizer.tokenize(word))>0] for t in tokens_2]

## Stemming

In [82]:
tokens_1 = [[porter.stem(word) for word in t] for t in tokens_1]
tokens_2 = [[porter.stem(word) for word in t] for t in tokens_2]

## Join the tokens to form a sentence

In [83]:
clean_review_1 = [" ".join(review) for review in tokens_1]
clean_review_2 = [" ".join(review) for review in tokens_2]

## Transform the text usinf vectorizer and make predictions

**NOTE: Prediction probability for positive class is high for the first test review**

In [84]:
X_test = vect.transform(clean_review_1)

In [85]:
X_test.shape

(1, 666)

In [86]:
model.predict_proba(X_test)

array([[0.00441452, 0.99558548]])

In [87]:
model.predict(X_test)

array([1])

**NOTE: Prediction probability for negative class is high for the second test review\**

In [88]:
X_test = vect.transform(clean_review_2)

In [89]:
X_test.shape

(1, 666)

In [90]:
model.predict_proba(X_test)

array([[0.87775701, 0.12224299]])

In [91]:
model.predict(X_test)

array([0])